# 중요 결정사항 및 설명란

# 필요 모듈

In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

from Fin import *
warnings.filterwarnings(action='ignore')

# 0. 엑셀읽기

In [2]:
df = pd.read_excel("Data/KospiPEISData&PriceData.xlsx")
df = Cut20NullfillNA(df)
# 여기서 deltaprice이상한거 제거하기

# 1. PEIS를 구하기 위한 기초데이터들 연산해보기 (기업별로)
* table 5로 기업별로 PEIS 기반 점수 구함

In [3]:
PEISBaseData = CompuMoreData(df)
PEISBaseData = setSGA(PEISBaseData)

In [4]:
PEISBaseData.shape

(8456, 45)

# Drop 시킬 데이터 코드모음 --> 나중에 deltaprice해야함
1. 일단 deltaprice 0 , 99 , inf 삭제
2. 년도 08, 09 ,20 ,21 삭제 20--> 이거는 나중에 가지고 있는 것도 좋을 것 같네
3. GNOA도 문제임 

* 원하는 결과 --> Null 값과 이상치가 없는게 여기서 이뤄져야함 그래야 PEIS점수 깔끔하게 구하기 가능 ㅇㅇ

# 매출액delta, 이상치 deltaGNOA문제 --> 둘다 inf있노;
0. 내 생각엔 기간을 2기간 잡아먹는 deltaATO에서 무조건 문제생김
1. inf의 경우 전년도가 매출액 0 이였음 --> 걍뺴자 inf
2, 금융권 친구들이 GNOA 에서 에러가 남 --> 금융권 비금융권 따로 계산해가지고 

In [5]:
dropdata(PEISBaseData, '회계년', [2010, 2011, 2020, 2021])
dropdata(PEISBaseData, 'deltaprice', [0, 99, np.inf])
dropdata(PEISBaseData, 'deltaACC', [np.inf, -np.inf])
dropdata(PEISBaseData, 'RNOA', [0]) # 그냥 연도 넘어가면서 이상해지는 것들 제거 
dropdata(PEISBaseData, 'SGAp', [np.inf, -np.inf]) # 그냥 매출액이 0인 경우도 있음.. 이건 뭐 어쩌란건지 원 
dropdata(PEISBaseData, 'deltaGM1', [np.inf, -np.inf]) # 그냥 매출액이 0인 경우도 있음.. 이건 뭐 어쩌란건지 원 
PEISBaseData.shape

(5793, 45)

# 2. 가공된 데이터 토대로 +1,-1,0 구하기 (년도별로)

In [6]:
Yearlydfs = yearbaseDFs(PEISBaseData, years =[2012,2013,2014,2015,2016,2017,2018,2019])
# --> years --> 10~19 까지만 사용 yeardata = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
PEISBaseData = CalPEISpoints(Yearlydfs)

# 3. PEIS 점수 구하기 (단순 행 sum)

In [7]:
PEISBaseData = CalPEIS(PEISBaseData)

In [18]:
test = PEISBaseData[['SGAdeltasale>0PEISpoint','SGAdeltasale<0PEISpoint','GNOAPEISpoint','deltaATOPEISpoint','deltaGMPEISpoint','deltaACCPEISpoint','RNOAPEISpoint']]!=0

In [19]:
test.sum()

SGAdeltasale>0PEISpoint    1400
SGAdeltasale<0PEISpoint     936
GNOAPEISpoint              2322
deltaATOPEISpoint          2322
deltaGMPEISpoint           2322
deltaACCPEISpoint          2322
RNOAPEISpoint              2322
dtype: int64

# 4. 저장하기

In [21]:
PEISBaseData.to_excel('Data/Alldata.xlsx', index = False)

# 4. 년도 별 포트폴리오 구하기 위해 년도 별로 나누기 -- 근데 이거는 포트폴리오 부분에서 따로하는게 좋을 듯

In [81]:
dfs = yearbaseDFs(PEISBaseData, years = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020])

## 5. 조건 & 년도별로 포트폴리오 수익률 구하기 (equally weight)

In [10]:
PEISbuyport = Upper20Port(dfs,"valPEIS")
np.mean(PEISbuyport)

0.12086963609933053

# 6. 엑셀로 보내기 

In [11]:
writer = pd.ExcelWriter('Result/Compute_PEIS.xlsx', engine='xlsxwriter')

In [12]:
# 이거 나중에 수정 필요
collist=['ADJprice', 'Consensus',\
       'deltaprice', 'CountNull', 'NetAsset', 'deltaACC', 'RNOA', 'GNOA',\
       'deltaGNOA', 'SGA', 'deltaSGA', 'GM', 'deltaGM1', 'deltasale',\
       'deltaGM2', 'ATO', 'deltaATO', 'SGAdeltasale>0', 'SGAdeltasale<0',\
       'GNOAPEISpoint', 'deltaATOPEISpoint', 'deltaGM2PEISpoint',\
       'deltaACCPEISpoint', 'RNOAPEISpoint', 'SGAdeltasale>0PEISpoint',\
       'SGAdeltasale<0PEISpoint', 'valPEIS']

In [13]:
Computed = PEISBaseData[collist]

In [14]:
PEISBaseData.to_excel(writer,sheet_name = 'All_data')
Computed.to_excel(writer,sheet_name = 'Computed')

In [15]:
writer.save()

# 이 아래에서는 이것저것 실험

In [149]:
pd.Series([1,23,321,42,3,1,5]).unique()

array([  1,  23, 321,  42,   3,   5], dtype=int64)

In [159]:
PEISBaseData.T.apply(lambda x : len(x.unique()), axis=1)

Name                 784
회계년                    8
Symbol               786
총자산(천원)             5793
총자산(평균)(천원)         5793
영업이익(천원)            5793
매출원가(천원)            5480
판매비와관리비(천원)         5793
금융원가(비영업)(천원)       5350
기타비용(비영업)(천원)       5791
보통주자본금(천원)          1921
유동부채(천원)            5421
비유동부채(천원)           5413
우선주자본금(천원)           246
현금및현금성자산(천원)        5793
단기사채(천원)             349
단기차입금(천원)           4423
매출액(천원)             5793
매출총이익(천원)           5793
영업활동으로인한현금흐름(천원)    5793
매출채권(천원)            5397
재고자산(천원)            5299
매입채무(천원)            5291
ADJprice            3490
Consensus             91
deltaprice          5633
consen_cut             5
CompuConsen            1
CountNull              3
NOA                 5793
AVGNOA              5793
RNOA                5793
GNOA                5793
deltaACC            5793
GM1                 5793
deltaGM1            5793
deltasale           5793
deltaGM             5491
ATO                 5793
deltaATO            5793


In [162]:
PEISBaseData['SGAdeltasale>0'].quantile(0.2)

-0.012982921235063024

In [169]:
np.percentile(PEISBaseData['SGAdeltasale>0'].unique(),0.2)

-1.2389015168630126

In [170]:
len(PEISBaseData['SGAdeltasale>0'])

5793